<a href="https://colab.research.google.com/github/LampsteR/Finance_Senitiments/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers peft accelerate bitsandbytes

import torch
from transformers import AutoModelForSequenceClassification
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from google.colab import drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import json

path = '/content/drive/MyDrive/merged_financial_sentiment.json'  # update path

with open(path, 'r', encoding='utf-8') as f:
    for _ in range(5):          # print first 5 entries
        line = f.readline()
        if not line:
            break
        record = json.loads(line)
        print(record)

{'text': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .', 'label': 1}
{'text': "For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .", 'label': 2}
{'text': 'In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .', 'label': 2}
{'text': 'Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales .', 'label': 2}
{'text': 'Operating profit totalled EUR 21.1 mn , up from EUR 18.6 mn in 2007 , representing 9.7 % of net sales .', 'label': 2}


In [4]:
# !pip install -q transformers

# import json
# from pathlib import Path

# import torch
# from transformers import AutoTokenizer

# # -------------------------------------------------------------------
# # 1. Read the newline-delimited JSON file (one record per line)
# # -------------------------------------------------------------------
# def load_jsonl(path):
#     path = Path(path)
#     records = []
#     with path.open("r", encoding="utf-8") as f:
#         for line in f:
#             if line.strip():
#                 records.append(json.loads(line))
#     return records

# records = load_jsonl("/content/drive/MyDrive/merged_financial_sentiment.json")

# # -------------------------------------------------------------------
# # 2. Normalize labels (string → int, sorted order: negative < neutral < positive)
# # -------------------------------------------------------------------
# label_order = ["negative", "neutral", "positive"]
# label_to_id = {label: idx for idx, label in enumerate(label_order)}

# def normalize_label(raw_label):
#     if isinstance(raw_label, str):
#         return label_to_id[raw_label.lower()]
#     return int(raw_label)

# texts = [record["text"] for record in records]
# labels = [normalize_label(record["label"]) for record in records]

# # -------------------------------------------------------------------
# # 3. Tokenize with FinBERT (padding/truncation to 256 tokens)
# # -------------------------------------------------------------------
# tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

# encodings = tokenizer(
#     texts,
#     padding="max_length",
#     truncation=True,
#     max_length=256,
#     return_tensors="pt"
# )

# # -------------------------------------------------------------------
# # 4. Keep only the tensors you need
# # -------------------------------------------------------------------
# dataset = {
#     "input_ids": encodings["input_ids"],
#     "attention_mask": encodings["attention_mask"],
#     "labels": torch.tensor(labels)
# }

# print(dataset["input_ids"].shape, dataset["attention_mask"].shape, dataset["labels"].shape)

In [5]:
  #  output_path = '/content/drive/MyDrive/Sentiment_tokens/finbert_inputs.pt'  # adjust folder/name
  #  torch.save(dataset, output_path)
  #  print(f'Saved tensors to {output_path}')

In [6]:
  !pip install -U bitsandbytes --extra-index-url https://download.pytorch.org/whl/cu124
  !pip show bitsandbytes

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
Name: bitsandbytes
Version: 0.48.2
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/bitsandbytes-foundation/bitsandbytes
Author: 
Author-email: Tim Dettmers <dettmers@cs.washington.edu>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: numpy, packaging, torch
Required-by: 


In [7]:
!pip install -q transformers peft accelerate bitsandbytes

import torch
from transformers import AutoModelForSequenceClassification
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from google.colab import drive

# --------------------------------------------------------------
# 0. Mount Drive once per session
# --------------------------------------------------------------
#drive.mount('/content/drive')

# --------------------------------------------------------------
# 1. Load the tokenized tensors you saved earlier
# --------------------------------------------------------------
dataset = torch.load('/content/drive/MyDrive/Sentiment_tokens/finbert_inputs.pt')
input_ids = dataset['input_ids']
attention_mask = dataset['attention_mask']
labels = dataset['labels']
num_labels = labels.unique().numel()

# --------------------------------------------------------------
# 2. Set up 4-bit (QLoRA) base model
# --------------------------------------------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
)

model = AutoModelForSequenceClassification.from_pretrained(
    'ProsusAI/finbert',
    num_labels=num_labels,
    quantization_config=bnb_config,
    device_map='auto',
)

# --------------------------------------------------------------
# 3. Configure and attach LoRA adapters
# --------------------------------------------------------------
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'value'],
    task_type='SEQ_CLS',
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# `model` is now ready for training with your tokenized tensors.

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


trainable params: 592,131 || all params: 110,076,678 || trainable%: 0.5379


In [8]:
# If you’re in a fresh runtime/cell, load the saved tensors again
dataset = torch.load('/content/drive/MyDrive/Sentiment_tokens/finbert_inputs.pt')
input_ids = dataset['input_ids']
attention_mask = dataset['attention_mask']
labels = dataset['labels']

class FinSentimentDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return self.labels.size(0)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx],
        }

train_dataset = FinSentimentDataset(input_ids, attention_mask, labels)

In [12]:
from torch.utils.data import TensorDataset
from transformers import Trainer, TrainingArguments

# Build the training dataset (uses the tensors loaded earlier)
train_dataset = FinSentimentDataset(input_ids, attention_mask, labels)

# Detect bf16 support (Ampere+ GPUs); fall back to fp16 otherwise
bf16_available = torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/finbert-qlora-checkpoints',
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    warmup_ratio=0.03,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    report_to='none',
    fp16=False ,
    bf16=False,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

ValueError: Attempting to unscale FP16 gradients.

In [14]:
!pip install -q transformers peft accelerate bitsandbytes

import torch
from transformers import AutoModelForSequenceClassification, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from google.colab import drive

drive.mount('/content/drive')

# Load saved tensors ----------------------------------------------------------
bundle = torch.load('/content/drive/MyDrive/Sentiment_tokens/finbert_inputs.pt')
input_ids = bundle['input_ids']
attention_mask = bundle['attention_mask']
labels = bundle['labels']
num_labels = labels.unique().numel()

# Build dict-returning dataset (avoids vars() error) --------------------------
class FinSentimentDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return self.labels.size(0)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx],
        }

train_dataset = FinSentimentDataset(input_ids, attention_mask, labels)

# Set up QLoRA model ----------------------------------------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
)

model = AutoModelForSequenceClassification.from_pretrained(
    'ProsusAI/finbert',
    num_labels=num_labels,
    quantization_config=bnb_config,
    device_map='auto',
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'value'],
    task_type='SEQ_CLS',
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

import os
from accelerate.state import AcceleratorState

AcceleratorState._reset_state()
os.environ["ACCELERATE_MIXED_PRECISION"] = "no"

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/finbert-qlora-checkpoints',
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    warmup_ratio=0.03,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    report_to='none',
    fp16=False,
    bf16=False,
    remove_unused_columns=False,
)

# Training hyperparameters (no mixed precision) -------------------------------
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/finbert-qlora-checkpoints',
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    warmup_ratio=0.03,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    report_to='none',
    fp16=False,
    bf16=False,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
trainable params: 592,131 || all params: 110,076,678 || trainable%: 0.5379


Step,Training Loss
50,1.770200
100,1.077500
150,0.844600
200,0.763900
250,0.708400
300,0.671300
350,0.607600
400,0.591900
450,0.591400
500,0.555200


TrainOutput(global_step=4311, training_loss=0.4594557790240786, metrics={'train_runtime': 2804.7326, 'train_samples_per_second': 24.59, 'train_steps_per_second': 1.537, 'total_flos': 9135797938191360.0, 'train_loss': 0.4594557790240786, 'epoch': 3.0})

In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel

# 1. Point to your saved adapter directory



adapter_dir = "/content/drive/MyDrive/finbert-qlora-checkpoints/checkpoint-4311"  # adjust if the folder is elsewhere

print("adapter_dir =", adapter_dir, type(adapter_dir))

import os
print("exists?", os.path.isdir(adapter_dir))

print("adapter_dir =", adapter_dir, type(adapter_dir))
# 2. Load tokenizer and label order
base_model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
label_names = ["negative", "neutral", "positive"]  # ensure this matches the order used during training

# 3. Load base FinBERT + LoRA adapter
base_model_name = "ProsusAI/finbert"
model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    num_labels=len(label_names),
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True
)
model = PeftModel.from_pretrained(model, adapter_dir)
model.eval()

# 4. Helper to score text
def classify_sentiment(texts):
    if isinstance(texts, str):
        texts = [texts]
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=256)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.inference_mode():
        logits = model(**inputs).logits
        probs = torch.nn.functional.softmax(logits, dim=-1)
        pred_ids = probs.argmax(dim=-1).cpu().tolist()
        predictions = [label_names[i] for i in pred_ids]
        confidences = probs.max(dim=-1).values.cpu().tolist()
    return list(zip(texts, predictions, confidences))

# 5. Try a few queries
queries = [
    "The company posted record profits and raised guidance.",
    "Regulators opened an investigation into the firm's accounting practices.",
    "Management expects performance to remain stable next quarter."
]

for text, label, confidence in classify_sentiment(queries):
    print(f"Text: {text}\nPrediction: {label} (confidence={confidence:.3f})\n")

adapter_dir = /content/drive/MyDrive/finbert-qlora-checkpoints/checkpoint-4311 <class 'str'>
exists? True
adapter_dir = /content/drive/MyDrive/finbert-qlora-checkpoints/checkpoint-4311 <class 'str'>


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Text: The company posted record profits and raised guidance.
Prediction: positive (confidence=0.991)

Text: Regulators opened an investigation into the firm's accounting practices.
Prediction: neutral (confidence=0.597)

Text: Management expects performance to remain stable next quarter.
Prediction: positive (confidence=0.993)

